In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
diabetes_data = load_diabetes()
X = diabetes_data.data
y = diabetes_data.target
num_samples = X.shape[0]  #number of samples
num_features = X.shape[1]  #number of features


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
# print(X_train.shape)
# actually you can do like this too
# X = np.insert(X, 0, 1, axis=1)
intercept = np.ones((X_train.shape[0], 1))
X_train   = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test    = np.concatenate((intercept, X_test), axis=1)
# print(X_train.shape)

(309, 10)
(309, 11)


array([[ 1.        ,  1.72744456, -0.90423839, ...,  0.80023299,
         0.77090906,  1.3952692 ],
       [ 1.        , -0.81900311,  1.10590306, ...,  1.60659854,
         0.80834772,  0.38300972],
       [ 1.        ,  0.90359385,  1.10590306, ..., -0.00613255,
         0.69353582,  0.12994485],
       ...,
       [ 1.        , -0.44452551, -0.90423839, ..., -1.61886365,
        -0.74545288, -0.88231462],
       [ 1.        , -1.04368967,  1.10590306, ...,  2.41296409,
         1.81611979,  0.8047845 ],
       [ 1.        ,  0.45422073,  1.10590306, ...,  0.80023299,
         1.0193482 ,  1.81704398]])

# Closed form solution Derivation

Gradient descent gives one possible mean for minimizing $J$, which uses iterative approach and may take time.  In the situation where we know that our cost function is strictly concave or convex, we can explicitly take its derivative to zero.  This process of such derivation is called obtaining the **normal equations** or **closed form**. 

The **closed form** of linear regression can be derived easily.  Let $\mathbf{X}$ be a matrix of shape $(m, n)$, $\boldsymbol{\theta}$ as shape $(n, )$, and $\mathbf{y}$ as vector of shape $(m, )$.  Instead of writing the cost function as power of square, we shall write it in matrix multiplication as follows:

$$\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}\boldsymbol{\theta} - \mathbf{y})^T*(\mathbf{X}\boldsymbol{\theta}-\mathbf{y})$$

Recall the following properties:

$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{X}^T\mathbf{X}=2\mathbf{X} \tag{A}$$
$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{A}\mathbf{X}=\mathbf{A}^T$$
$$(\mathbf{X}\mathbf{y})^T = \mathbf{y}^T\mathbf{X}^T$$

Therefore

\begin{align*}
\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}\boldsymbol{\theta} - \mathbf{y})^T*(\mathbf{X}\boldsymbol{\theta}-\mathbf{y}) &= \frac{\partial J}{\partial \boldsymbol{\theta}} (\boldsymbol{\theta}^T\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \boldsymbol{\theta}^T\mathbf{X}^T\mathbf{y} - \mathbf{y}^T\mathbf{X}\boldsymbol{\theta} + \mathbf{y}^T\mathbf{y})\\
&= 2\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - 2\mathbf{X}^T\mathbf{y} \tag{see note*}\\
\end{align*}

Now, we can set the derivative to 0 to find out the optimal theta

$$\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \mathbf{X}^T\mathbf{y} = 0$$

Solving this gives us

$$\boldsymbol{\theta} =  (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$


Note*: Since $\mathbf{X}\boldsymbol{\theta}$ is a vector, and so is $\mathbf{y}$, it doesn't matter what the order is, thus we can simply add them to 2.  Also, we got 2 in front of the first part because we have two $\theta$ (used the property A)


**Why not closed form always**.  The answer is simple.  It does not always exists or possible, for example, the cost function is not convex or concave.  But of course, if it exists, we usually prefer closed form given that it is usually faster than gradient descent.  Nevertheless, as you can see, taking inverse of huge number of features can be expensive, thus it is also not always straightforward thing to always prefer closed form.

Yes, that's it for most of the theoretical stuff.  Let's start implementing some of these concepts so we can better understand them.

The closed form is a normal equations derived from setting the derivatives = 0.  By performing only some inverse operations and matrix multiplication, we will be able to get the theta.

$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

When closed form is available, is doable (can be inversed - can use pseudoinverse), and with not many features (i.e., inverse can be slow), it is recommended to always use closed form.  

## Implementation steps:

1. Prepare your data
    - add intercept
    - $\mathbf{X}$ and $\mathbf{y}$ and $\mathbf{w}$ in the right shape
        - $\mathbf{X}$ -> $(m, n)$
        - $\mathbf{y}$ -> $(m, )$
        - $\mathbf{w}$ -> $(n, )$
        - where $m$ is number of samples
        - where $n$ is number of features
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Plug everything into the equation.  Here we shall use X_train to retrieve the $\boldsymbol{\theta}$
$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

1. We simply using the $\boldsymbol{\theta}$, we can perform a dot product with our X_test which will give us $\mathbf{\hat{y}}$.

2. We then calculate the errors using mean-squared-error function:

$$\frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})^2$$

Note that it's a bit different from our $J(\boldsymbol{\theta})$ because $J(\boldsymbol{\theta})$ puts $\frac{1}{2}$ instead of $\frac{1}{m}$ for mathematical convenience for derivatives, since we know changing constants do not change the optimization results.


Let's implement.

In [28]:
from numpy.linalg import inv
class LinearRegression:
    def __init__(self, num_features):
        self.W = np.zeros(num_features)
    
    def fit(self, X, y):
        self.W = inv(X.T @ X) @ X.T @ y
        return self.W
    
    def predict(self, X):
        y_pred = X @ self.W
        return y_pred
    
    def eval(self, y_true, y_pred):
        assert y_true.shape == y_pred.shape
        mse = ((y_true - y_pred)**2).sum() / len(y_train)
        print("Mean squared errors: ", mse)
        return mse
    

In [32]:
model = LinearRegression(num_features=num_features)
W_star = model.fit(X=X_train, y=y_train)
y_pred = model.predict(X=X_test)
# print(y_pred.shape)
# print(y_test.shape)
model.eval(y_test,
           y_pred)

Mean squared errors:  1290.8513558924972


1290.8513558924972